In [1]:
import os

In [2]:
%pwd

'D:\\languages\\python\\MLOps\\diabetes_classification_project\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'D:\\languages\\python\\MLOps\\diabetes_classification_project'

In [5]:
# Now that we are in the directory to work and import the functions from src, let's create a @dataclass

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # this means this class will be immutable
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from diabetes_classification_project.constants import *
from diabetes_classification_project.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from diabetes_classification_project import logger
from diabetes_classification_project.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-04 17:29:06,836: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-04 17:29:06,885: INFO: common: yaml file: param.yaml loaded successfully]
[2024-05-04 17:29:06,900: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-04 17:29:06,900: INFO: common: created directory at: artifacts]
[2024-05-04 17:29:06,900: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-04 17:29:10,131: INFO: 1268487314: artifacts/data_ingestion/diabetes_dataset.zip download! with following info: 
Connection: close
Content-Length: 9128
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "56f901e3922e443b1062af733008c034b43501e3e89eedab5f91e272569e8f63"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8D54:250FB7:989D7:D4EB0:6636230D
Accept-Ranges: bytes
Date: